# What students should learn
1. How to build a basic CNN with guard rails.
2. How to take into account dimensions when performing pooling
3. Data augmentation

In [124]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import inspect

In [3]:
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                            train=False, 
                            transform=transforms.ToTensor(),
                            download = True)
# train_dataset = dsets.MNIST(root='./data', 
#                             train=True, 
#                             transform=transforms.Compose([
#                                 transforms.RandomRotation(degrees = 10),
#                                 transforms.ToTensor(),
#                                 ]),
#                             download=True)

# test_dataset = dsets.MNIST(root='./data', 
#                             train=False, 
#                             transform=transforms.Compose([
#                                 transforms.RandomRotation(degrees = 10),
#                                 transforms.ToTensor(),
#                                 ]),
#                             download = True)

Processing...
Done!


In [4]:
print(train_dataset.train_data.size())
print(train_dataset.train_labels.size())

torch.Size([60000, 28, 28])
torch.Size([60000])


In [5]:
print(test_dataset.test_data.size())
print(test_dataset.test_labels.size())

torch.Size([10000, 28, 28])
torch.Size([10000])


In [6]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [75]:
nn.modules.ReLU

In [155]:
class CNNModel(nn.Module):
    def __init__(self, in_channels_list = [1, 16], out_channels_list = [16, 32],
                 kernel_size_list = [5, 5], stride_list = [1, 1],
                 padding_list = [0,0], kernel_pool_list = [2, 2],
                 pooling_list = [nn.MaxPool2d, nn.MaxPool2d], activations_list = [nn.ReLU(), nn.ReLU()]):
        localArgs = locals().items()
        super(CNNModel, self).__init__()
        argLens = set()
        ignoredArgs = ['self', "__class__"]
        for argName, arg in localArgs:
            if argName not in ignoredArgs:
                argLens.add(len(arg))
        assert len(argLens) == 1, ("mismatch in lengths of arguments."
                                   "All params for each layer must be specified")
#         listLengths = np.array([len()])
        # Convolution 1
        modules = list()
        for layerIdx in range(0, argLens.pop()):
            modules.append(nn.Conv2d(in_channels = in_channels_list[layerIdx],
                                 out_channels = out_channels_list[layerIdx],
                                 kernel_size = kernel_size_list[layerIdx],
                                 stride = stride_list[layerIdx],
                                 padding = padding_list[layerIdx]))
            modules.append(activations_list[layerIdx])
            modules.append(pooling_list[layerIdx](kernel_size = kernel_pool_list[layerIdx]))
        self.convolutions = nn.Sequential(*modules)
        self.finalLayer = nn.Linear(32 * 4 * 4, 10)

    def forward(self, x):
        out = self.convolutions(x)
        out = out.view(out.size(0), -1)
        out = self.finalLayer(out)
        return(out)

In [157]:
model = CNNModel()

In [158]:
criterion = nn.CrossEntropyLoss()

In [159]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [160]:
for modelParam in model.parameters():
    print(modelParam.size())
print(model.parameters())

print(len(list(model.parameters())))

torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 512])
torch.Size([10])
<generator object Module.parameters at 0x7f92bddcde08>
6


In [ ]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 50. Loss: 2.2498464584350586. Accuracy: 18
Iteration: 100. Loss: 2.132849931716919. Accuracy: 58
Iteration: 150. Loss: 1.7954744100570679. Accuracy: 66
Iteration: 200. Loss: 1.2130354642868042. Accuracy: 77
Iteration: 250. Loss: 0.7746879458427429. Accuracy: 83
Iteration: 300. Loss: 0.5422929525375366. Accuracy: 85
Iteration: 350. Loss: 0.6932182312011719. Accuracy: 86
Iteration: 400. Loss: 0.37876656651496887. Accuracy: 87
Iteration: 450. Loss: 0.5879152417182922. Accuracy: 86
Iteration: 500. Loss: 0.5827994346618652. Accuracy: 88
Iteration: 550. Loss: 0.23624227941036224. Accuracy: 90
Iteration: 600. Loss: 0.31086039543151855. Accuracy: 90
Iteration: 650. Loss: 0.351461261510849. Accuracy: 90
Iteration: 700. Loss: 0.3782237768173218. Accuracy: 91
Iteration: 750. Loss: 0.3811475336551666. Accuracy: 91
Iteration: 800. Loss: 0.3364792764186859. Accuracy: 91
Iteration: 850. Loss: 0.41134724020957947. Accuracy: 92
Iteration: 900. Loss: 0.26772287487983704. Accuracy: 93
Iteratio